In [ ]:
import os
import requests
import pandas as pd
from datetime import datetime, timedelta
from dotenv import load_dotenv
import time
import json

# Cargar la clave API desde el archivo .env
# Porque guardar claves directamente en el código es para quienes disfrutan sufrir públicamente.
load_dotenv()
API_KEY = os.getenv("FMP_key")
if not API_KEY:
    raise SystemExit("❌ No se encontró FMP_KEY en el .env (sorpresa: sin eso, nada funciona)")

# Lista de tickers del Nasdaq: básicamente los Avengers del mercado.
tickers_nasdaq = ['NVDA', 'MSFT', 'AAPL', 'AMZN', 'META', 'GOOG', 'GOOGL', 'TSLA', 'AVGO', 'NFLX']

# Calculamos el rango de fechas: últimos 90 días, porque mirar más atrás ya es arqueología.
hoy = datetime.today()
fecha_limite = hoy - timedelta(days=90)
fecha_desde = fecha_limite.strftime('%Y-%m-%d')
fecha_hasta = hoy.strftime('%Y-%m-%d')

def descargar_eod_fmp(symbol):
    """
    Descarga los datos históricos diarios de un ticker desde FMP.
    Incluye precios ajustados por splits y dividendos (como si entendieras lo que eso implica).
    """
    # El endpoint de la API, también conocido como “donde tus esperanzas serán evaluadas por el servidor”.
    url = (
        f"https://financialmodelingprep.com/api/v3/historical-price-full/{symbol}"
        f"?from={fecha_desde}&to={fecha_hasta}&apikey={API_KEY}"
    )
    
    try:
        # Enviamos la petición a internet, donde los sueños van a morir lentamente.
        r = requests.get(url, timeout=15)
        r.raise_for_status()  # Si algo va mal, Python se quejará con estilo.
        data = r.json()
        
        # A veces FMP responde con cosas raras. Esto lo comprueba.
        if not isinstance(data, dict):
            print(f"⚠️ {symbol}: Respuesta inesperada (no es un diccionario, probablemente una decepción)")
            return None
        
        # Si FMP está de mal humor, nos lo dice aquí.
        if "Error Message" in data:
            print(f"❌ {symbol}: {data['Error Message']}")
            return None
        
        # Los datos vienen en la clave ‘historical’, si no, algo salió horriblemente mal.
        if 'historical' not in data:
            print(f"⚠️ {symbol}: No se encontró clave 'historical'.")
            print(f"   Claves disponibles: {list(data.keys())}")
            print(f"   Contenido: {json.dumps(data, indent=2)[:300]}")  # Mostramos una pizca del desastre.
            return None
        
        historical_data = data['historical']
        
        # A veces, los datos simplemente... no existen. Como tu motivación un lunes.
        if not historical_data or len(historical_data) == 0:
            print(f"⚠️ {symbol}: No hay datos históricos para el periodo solicitado (FMP decidió ignorarte).")
            return None
        
        # Creamos un DataFrame con los datos, porque usar Excel manualmente es para masoquistas.
        df = pd.DataFrame(historical_data)
        
        # Renombramos columnas porque FMP cree en el caos y la inconsistencia.
        df = df.rename(columns={
            'date': 'date',
            'open': 'open',
            'high': 'high',
            'low': 'low',
            'close': 'close',
            'adjClose': 'adj_close',
            'volume': 'volume',
            'unadjustedVolume': 'unadjusted_volume',
            'change': 'change',
            'changePercent': 'change_percent',
            'vwap': 'vwap',
            'label': 'label',
            'changeOverTime': 'change_over_time'
        })
        
        # Convertimos la columna de fecha al formato datetime, porque las cadenas de texto no hacen cálculos.
        df['date'] = pd.to_datetime(df['date'])
        
        # Añadimos el símbolo al DataFrame, por si luego se te olvida de dónde vino cada dato (lo hará).
        df['symbol'] = symbol
        
        # Ordenamos por fecha ascendente: del pasado al presente, como una persona razonable.
        df = df.sort_values('date')
        
        # Seleccionamos las columnas que realmente importan (el resto solo ocupa espacio como tus fotos duplicadas).
        columnas_principales = ['date', 'symbol', 'open', 'high', 'low', 'close', 'adj_close', 'volume']
        df = df[columnas_principales]
        
        print(f"✅ {symbol}: {len(df)} filas descargadas ({df['date'].min().date()} a {df['date'].max().date()})")
        
        return df
        
    except requests.exceptions.HTTPError as e:
        print(f"❌ {symbol}: Error HTTP {e.response.status_code} (el servidor dijo: 'no').")
        if e.response.status_code == 401:
            print("   Tu API key no sirve. Intenta rezar.")
        elif e.response.status_code == 403:
            print("   Límite de requests alcanzado. FMP te odia hoy.")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ {symbol}: Error de conexión - {e} (¿apagaste el WiFi otra vez?)")
        return None
    except json.JSONDecodeError as e:
        print(f"❌ {symbol}: Error al parsear JSON - {e} (FMP te envió basura).")
        return None
    except KeyError as e:
        print(f"❌ {symbol}: Falta columna esperada - {e} (gracias, FMP, por ser tan creativo).")
        return None
    except Exception as e:
        print(f"❌ {symbol}: Error inesperado - {e} (plot twist del código).")
        return None

def main():
    """Función principal para descargar todos los tickers — también conocida como ‘la parte que más falla’."""
    print("="*60)
    print("DESCARGA DE DATOS HISTÓRICOS - FINANCIAL MODELING PREP")
    print("="*60)
    print(f"Periodo: {fecha_desde} a {fecha_hasta}")
    print(f"Tickers: {', '.join(tickers_nasdaq)}")
    print(f"Total de tickers: {len(tickers_nasdaq)}")
    print("="*60)
    
    todo = []     # Aquí irán los DataFrames exitosos.
    errores = []  # Aquí tus fracasos, digo... tickers fallidos.
    
    for i, ticker in enumerate(tickers_nasdaq):
        print(f"\n[{i+1}/{len(tickers_nasdaq)}] Descargando {ticker}...")
        
        df = descargar_eod_fmp(ticker)
        
        if df is not None:
            todo.append(df)
        else:
            errores.append(ticker)
        
        # FMP te deja hacer 4 requests por segundo. Dormimos 0.3 seg para no cabrear al servidor.
        if i < len(tickers_nasdaq) - 1:
            time.sleep(0.3)
    
    # Resumen tipo “esto es lo que lograste, campeón”.
    print("\n" + "="*60)
    print("RESUMEN DE DESCARGA")
    print("="*60)
    print(f"✅ Exitosos: {len(todo)}/{len(tickers_nasdaq)}")
    if errores:
        print(f"❌ Con errores: {len(errores)} - {', '.join(errores)} (momentos de silencio, por favor)")
    
    # Si logramos algo, combinamos todo en un DataFrame gigante, porque el éxito es compartir.
    if todo:
        df_all = pd.concat(todo, ignore_index=True)
        df_all = df_all.sort_values(['symbol', 'date'])
        
        # Guardamos los datos en CSV, el formato universal para gente que no usa bases de datos.
        archivo_salida = "nasdaq_eod_3meses_fmp.csv"
        df_all.to_csv(archivo_salida, index=False)
        
        print(f"\n✅ Datos guardados en: {archivo_salida}")
        print(f"   Total de filas: {len(df_all):,}")
        print(f"   Rango de fechas: {df_all['date'].min().date()} a {df_all['date'].max().date()}")
        print(f"   Símbolos únicos: {df_all['symbol'].nunique()}")
        
        # Mostramos un adelanto, para que veas que realmente hicimos algo.
        print("\n📊 Primeras filas del dataset:")
        print(df_all.head(10).to_string(index=False))
        
        # Y aquí las estadísticas que nadie leerá pero se ven profesionales.
        print("\n📈 Estadísticas por ticker:")
        stats = df_all.groupby('symbol').agg({
            'date': ['count', 'min', 'max'],
            'close': ['mean', 'std'],
            'volume': 'mean'
        }).round(2)
        print(stats)
        
        return df_all
    else:
        # Si todo falló, al menos te lo decimos con empatía robótica.
        print("\n❌ No se pudo descargar ningún dato.")
        print("   Verifica:")
        print("   1. Tu API key es válida (spoiler: no lo es).")
        print("   2. No has excedido el límite gratuito (sí lo hiciste).")
        print("   3. Los tickers existen (quizás no).")
        return None

# Entrada triunfal del programa: el equivalente a pulsar “ejecutar” con esperanza.
if __name__ == "__main__":
    df_final = main()